### Validation Gender optimiert

In [1]:
import pandas as pd
import numpy as np

#df_test = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/validation_test/test_final_FeaturesExtracted.csv', delimiter = ';')
df_test = pd.read_csv ('/home/ec2-user/SageMaker/voice_gender_age/validation_test/FeaturesExtracted_TestdatenBDC_Gender_Age.csv', delimiter = ';')

list_cn = df_test.columns.values
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn = np.delete(list_cn,0)
list_cn

nrows = len(df_test)

for i in range (0,nrows):
    for j in list_cn:
        list_c = df_test[j].iloc[i]
        list_c = list_c.replace('[','')
        list_c = list_c.replace(']','')
        list1 = np.fromstring (list_c, sep=' ',dtype=float)
        df_test[j].iat[i] = list1
        df_test[j].iat[i] = df_test[j].iloc[i].mean()
    if (i % 10000 == 0):
        print ("Reihe "+ str(i) + " fertig")
print ("Konvertierung abgeschlossen.")

print (df_test.shape)

df_pred = df_test.copy()
del df_pred['Unnamed: 0']
del df_pred['index']
del df_pred['path']
del df_pred['age']
del df_pred['file_wave']
del df_pred['gender']

#list (df_pred.columns)
#df_pred.shape

print ("Fertig !!!")

Reihe 0 fertig
Konvertierung abgeschlossen.
(11, 40)
Fertig !!!


In [2]:
import tensorflow as tf
from tensorflow import keras
# model_age = keras.models.load_model ('/home/ec2-user/SageMaker/voice_gender_age/Age_without_Gender_opti/model.h5', custom_objects = None, compile = True)
# Neues Modell mit breiter Spanne
# model_age = keras.models.load_model ('/home/ec2-user/SageMaker/voice_gender_age/Age_without_Gender_opti/model_breiteSpanne.h5', custom_objects = None, compile = True)
model_gender_opt = keras.models.load_model ('/home/ec2-user/SageMaker/voice_gender_age/Optimierung_Gender_UC1/model.h5', custom_objects = None, compile = True)

# model_age.summary()

df_final = df_test.copy()
df_final['pred_class']=0
df_final['pred_proba']=0

for i in range (7):
    prediction = model_gender_opt.predict(df_pred.iloc[[i]])
    prediction_class = model_gender_opt.predict_classes(df_pred.iloc[[i]])
    prediction_proba = model_gender_opt.predict_proba(df_pred.iloc[[i]])
    print(prediction)
    print (prediction_class)
    df_final.at[i,'pred_class'] = prediction_class
    #df_final.at[i,'pred_proba'] = prediction_proba
    #print (prediction_proba)
    print ("----------")
    
anzahl_samples = df_final.shape[0]
korrekt = 0
falsch = 0


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[[0.9808297]]
[[1]]
----------
[[0.9653391]]
[[1]]
----------
[[0.99986136]]
[[1]]
----------
[[0.99682057]]
[[1]]
----------
[[0.28632566]]
[[0]]
----------
[[0.9284343]]
[[1]]
----------
[[0.94071114]]
[[1]]
----------


0 = male 1 = female

In [3]:
## Mit großer Spanne
for i in range (anzahl_samples):
    if ((df_final.pred_class[i]==0) & (df_final.gender[i] == 'male')):
        korrekt +=1
    elif ((df_final.pred_class[i]==1) & (df_final.age[i] == 'female')):
        korrekt +=1
    else:
        falsch +=1

print ('Korrekt: ' + str(korrekt) +'/'+ str(anzahl_samples))
print ('Falsch: ' + str(falsch) +'/'+ str(anzahl_samples))

Korrekt: 4/11
Falsch: 7/11
